## RNN Character Generator

Use RNN to generate a play. Let RNN leatn from an example and want it will learn how to write a version of its own. Using a character predictive model that will take as input a variable length sequence and predict the next character.

In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


### Dataset
For this example, we only need one peice of training data. In fact, we can write our own poem or play and pass that to the network for training if we'd like. However, to make things easy we'll use an extract from a shakesphere play.

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


### Loading Your Own Data
To load your own data, you'll need to upload a file from the dialog below. Then you'll need to follow the steps from above but load in this new file instead.



In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

IndexError: ignored

### Read Contents of File
Let's look at the contents of the file.

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Encoding
Since this text isn't encoded yet we'll need to do that ourselves. We are going to encode each unique character as a different integer.



In [ ]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


And here we will make a function that can convert our numeric values to text.


In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Creating Training Examples
Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we will prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Our first step will be to create a stream of characters from our text data.

In [ ]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next we can use the batch method to turn this stream of characters into batches of desired length.

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Now we need to use these sequences of length 101 and split them into input and output.

In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


Finally we need to make training batches.

In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Building the Model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


### Creating Loss Function
Create our loss function for the problem. The model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.

However, before we do that let's have a look at a sample input and the output from our untrained model. This is so we can understand what the model is giving us.



In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 5.90070849e-04  1.45298510e-03  8.95539299e-03 ...  3.88986897e-04
    1.79225416e-03  7.17221433e-03]
  [-4.85264463e-03  3.49858403e-03  9.61992703e-03 ...  3.92884342e-03
   -1.77465053e-03  7.18819909e-03]
  [-2.74801999e-03  4.31347918e-03  1.13415741e-03 ...  1.55285764e-02
   -1.69506459e-03  3.90203740e-03]
  ...
  [-8.81693698e-03  1.96716934e-03 -9.99123137e-03 ...  9.97600146e-03
   -1.33429952e-02  4.04036371e-03]
  [-7.37762591e-03  3.46151111e-03 -6.90134987e-03 ...  1.16449818e-02
   -7.36044580e-03  3.53406440e-03]
  [-5.75478189e-03  4.99309273e-03 -4.27727867e-03 ...  1.34483669e-02
   -3.20540881e-03  3.59972427e-03]]

 [[ 2.28316546e-03  1.15239043e-02  4.59780218e-03 ... -9.13207419e-04
    5.87319396e-03 -2.20199363e-04]
  [-2.49909295e-04  1.62480325e-02 -2.37524952e-03 ...  5.76629769e-03
    5.58166904e-03  7.80398585e-03]
  [ 1.00615993e-03  1.33211296e-02 -7.17280339e-03 ...  1.63274892e-02
    3.37955588e-03  5.02782781e-03]
  ...
  [ 9.334

In [ ]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00059007  0.00145299  0.00895539 ...  0.00038899  0.00179225
   0.00717221]
 [-0.00485264  0.00349858  0.00961993 ...  0.00392884 -0.00177465
   0.0071882 ]
 [-0.00274802  0.00431348  0.00113416 ...  0.01552858 -0.00169506
   0.00390204]
 ...
 [-0.00881694  0.00196717 -0.00999123 ...  0.009976   -0.013343
   0.00404036]
 [-0.00737763  0.00346151 -0.00690135 ...  0.01164498 -0.00736045
   0.00353406]
 [-0.00575478  0.00499309 -0.00427728 ...  0.01344837 -0.00320541
   0.00359972]], shape=(100, 65), dtype=float32)


In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 5.9007085e-04  1.4529851e-03  8.9553930e-03  3.7790635e-03
 -2.7456635e-03 -1.2415566e-03 -2.2686203e-03  3.0455019e-03
  4.1467259e-03  2.9484534e-03 -3.8900308e-03  1.3582340e-03
 -2.4997832e-03  4.6997247e-03 -1.7394401e-03 -2.9325741e-03
  3.1141504e-03  1.2567646e-03  5.2182213e-03  6.7870980e-03
  2.2131857e-04  2.9783982e-03  1.8283050e-04 -3.2337109e-04
  6.1952532e-04 -7.0371851e-04  1.2474926e-04  3.4127538e-03
 -1.8665316e-03  4.0747654e-03 -9.2380750e-04 -5.5821026e-03
 -8.9108484e-04 -3.2068046e-03 -4.6935165e-05 -5.7007056e-03
 -1.0777300e-03 -2.3713396e-03  7.6444040e-04 -2.3612354e-03
  2.2780290e-04 -1.5234045e-03 -9.8974048e-04  3.0707736e-03
 -2.1567324e-03 -1.1377921e-04 -2.9534609e-03  1.4850575e-03
  5.0620544e-03 -3.6494923e-04  3.1988067e-03 -4.9512601e-03
  2.9751513e-04  2.1702601e-03  4.6173781e-03 -1.0768330e-03
 -1.5325806e-03 -4.4289050e-03  9.8253484e-05 -2.6558526e-05
 -5.6766565e-03  2.1778091e-04  3.8898690e-04  1.7922542e-03
  7.172214

In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"NNcXMl.QV3oKUJAiVHEf;HZC,\nOmyfJDYf3kjqbuuQuzQ ;jNnwiwKQL\n dnIWfxTms,Mo'H,;qy:MQtt!ZZaZWlWGSqeK!a,Dwp"

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were. 

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

### Compiling the Model

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Creating Checkpoints
Setup and configure our model to save checkpoinst as it trains. This will allow to load our model from a checkpoint and continue training it.

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Training the model. 

In [ ]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 14s 63ms/step - loss: 2.6151
Epoch 2/50
172/172 [==============================] - 12s 63ms/step - loss: 1.8975
Epoch 3/50
172/172 [==============================] - 12s 63ms/step - loss: 1.6491
Epoch 4/50
172/172 [==============================] - 12s 64ms/step - loss: 1.5142
Epoch 5/50
172/172 [==============================] - 12s 65ms/step - loss: 1.4317
Epoch 6/50
172/172 [==============================] - 13s 66ms/step - loss: 1.3747
Epoch 7/50
172/172 [==============================] - 13s 67ms/step - loss: 1.3300
Epoch 8/50
172/172 [==============================] - 13s 67ms/step - loss: 1.2902
Epoch 9/50
172/172 [==============================] - 13s 68ms/step - loss: 1.2558
Epoch 10/50
172/172 [==============================] - 13s 68ms/step - loss: 1.2214
Epoch 11/50
172/172 [==============================] - 13s 69ms/step - loss: 1.1869
Epoch 12/50
172/172 [==============================] - 13s 71ms/step - loss: 1.1509
E

### Loading the Model
Rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one peice of text to the model and have it make a prediction.

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model is finished training, we can find the **lastest checkpoint** that stores the models weights using the following line.



In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

load **any checkpoint** we want by specifying the exact file to load.

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

AttributeError: ignored

### Generating Text
Now we can use the lovely function provided by tensorflow to generate some text using any starting string we'd like.

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: let me try
let me try.

KING RICHARD III:
Harp not on that?

LADY CAPULET:
We shall be shorten'd in a happy torns:
Ah, God defend the gods, black-past, let them not.

EXETER:
This way the friar of dark commends them thence,
So sluck him deadly to my brother too,
Nor much for love with no less to besiege us.

First Murderer:
How came your king unbrothers' bosoms!

WARWICK:
Tell her a-to consider in this wide,
How is it more, withal where the other sleith.

DUKE VINCENTIO:
Could you not have it you.

STANLEY:
My lord, in Yorkshire are in safety through the man
As hazer mercy as hast forswear him, ne'er
Till Richard Stanley did remembere his majesty
To save this birth deliver degrees withal
Kings and power i' the freedom of my knield sight!
For shaming comely face to follow him,
As he hath followed you, with an h
